# Scrapping Tailwindcss

Scrapping simple de la página de [Tailwindcss](https://tailwindcss.com/docs/customizing-colors) para obtener los colores y sus gradientes. Se hará la extracción del "nombre" del set de colores, el "tramo" y el "color" en formato hexadecimal.

## Librerías necesarias

```python
import requests
from bs4 import BeautifulSoup
import json
```

## Resultado esperado

```json
[
  {
    "name": "Slate",
    "data": [
      { "tramo": "50", "color": "#f8fafc" },
      { "tramo": "100", "color": "#f1f5f9" },
      { "tramo": "200", "color": "#e2e8f0" },
      { "tramo": "300", "color": "#cbd5e1" },
      { "tramo": "400", "color": "#94a3b8" },
      { "tramo": "500", "color": "#64748b" },
      { "tramo": "600", "color": "#475569" },
      { "tramo": "700", "color": "#334155" },
      { "tramo": "800", "color": "#1e293b" },
      { "tramo": "900", "color": "#0f172a" },
      { "tramo": "950", "color": "#020617" }
    ]
  },
  // ...
]
```

### Importamos las librerías necesarias

In [1]:
# IMPORT Libraries
import requests # to get the html source code of the page
import  bs4 # to parse the html source code
import json # to convert the data into json format
URL = 'https://tailwindcss.com/docs/customizing-colors'

# output file: path to save the json file
OUTPUT_FILE = 'data/data.json'

### Extrayendo los Tags que contengan estas clases
```js
====== General Arrays ======
COLORS INFO -----    "grid mt-3 grid-cols-1 sm:grid-cols-11 gap-y-3 gap-x-2 sm:mt-2 2xl:mt-0"
NAMES       -----    "text-sm font-semibold text-slate-900 dark:text-slate-200 2xl:col-end-1 2xl:pt-2.5"

=== For each tramo/color ===
-- COLORS   -----    "w-6 font-medium text-xs text-slate-900 2xl:w-full dark:text-white"
-- TRAMOS   -----    "text-slate-500 text-xs font-mono lowercase dark:text-slate-400 sm:text-[0.625rem] md:text-xs lg:text-[0.625rem] 2xl:text-xs"
```

In [2]:
# ====== General Arrays ======
search_array_colors = "grid mt-3 grid-cols-1 sm:grid-cols-11 gap-y-3 gap-x-2 sm:mt-2 2xl:mt-0"
search_array_names = "text-sm font-semibold text-slate-900 dark:text-slate-200 2xl:col-end-1 2xl:pt-2.5"

# === For each tramo/color ===
search_array_colors_info_tramos = "w-6 font-medium text-xs text-slate-900 2xl:w-full dark:text-white"
search_array_colors_info_color = "text-slate-500 text-xs font-mono lowercase dark:text-slate-400 sm:text-[0.625rem] md:text-xs lg:text-[0.625rem] 2xl:text-xs"

### Extraemos y parseamos los datos

In [3]:
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text,'html.parser')

div_color_set_information = soup.find_all('div', class_ = search_array_colors)
div_color_set_names = soup.find_all('div', class_ = search_array_names)

In [4]:
color_set_names = [name.text for name in div_color_set_names]
color_set_names [:5]

['Slate', 'Gray', 'Zinc', 'Neutral', 'Stone']

In [5]:
# data will contain 
# 
# [{
#     "name": "Slate",
#     "data": [
#       { "tramo": "50", "color": "#f8fafc" },
#       // ...    
#  ]}
#  ]
data = []

### Extraemos información para cada color

In [6]:
# Searching for the "tramo" and "color" for each color set

each_color_information = []
for i in range(len(div_color_set_information)):
    name = color_set_names[i] # ex: "Slate"
    div_info_color = div_color_set_information[i].find_all('div',class_=search_array_colors_info_color) # ex: [<div>...</div>, <div>...</div>, ...]

    for index, current_tramo in enumerate(div_color_set_information[i].find_all('div',class_=search_array_colors_info_tramos)):
        # Get "tramo" and "color" for each color set
        tramo = current_tramo.text # ex: "50"
        color = div_info_color[index].text # ex: "#f8fafc"
        each_color_information.append({
            'tramo': tramo,
            'color': color
        })
    
    # Append the data to the main array
    data.append({
        'name': name,
        'data': each_color_information
    })

    # Reset the array for the next color set
    each_color_information = []

# Save as .json
# Replace the file if it already exists

with open(OUTPUT_FILE, 'w') as outfile:
    json.dump(data, outfile)

data = []